## Task 1. Data Quality Assessment

In [3]:
# Lets import libraries

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [10]:
# Lets find out what worksheets are in provided excel file

import xlrd
book = xlrd.open_workbook("KPMG.xlsx")
print("The number of worksheets is {0}".format(book.nsheets))
print("Worksheet name(s): {0}".format(book.sheet_names()))


The number of worksheets is 5
Worksheet name(s): ['Title Sheet', 'Transactions', 'NewCustomerList', 'CustomerDemographic', 'CustomerAddress']


In [159]:
# Read in the xlsx file each of 3 required sheets into separate variables

transactions = pd.read_excel("KPMG.xlsx", 'Transactions', header = 1)
customerdemo = pd.read_excel("KPMG.xlsx", 'CustomerDemographic', header = 1)
customeradd = pd.read_excel("KPMG.xlsx", 'CustomerAddress', header = 1)

1_1. Lets start looking inside transactions data. By describe() and summing NaN we'll see missing info in each columns if any

In [162]:
# See the quantitative summary of transactions
print("transactions quantity {} \n".format(len(transactions)))
print(transactions.describe())
print('\n',transactions.head())

transactions quantity 20000 

       transaction_id   product_id   customer_id  online_order    list_price  \
count    20000.000000  20000.00000  20000.000000  19640.000000  20000.000000   
mean     10000.500000     45.36465   1738.246050      0.500458   1107.829449   
std       5773.647028     30.75359   1011.951046      0.500013    582.825242   
min          1.000000      0.00000      1.000000      0.000000     12.010000   
25%       5000.750000     18.00000    857.750000      0.000000    575.270000   
50%      10000.500000     44.00000   1736.000000      1.000000   1163.890000   
75%      15000.250000     72.00000   2613.000000      1.000000   1635.300000   
max      20000.000000    100.00000   5034.000000      1.000000   2091.470000   

       standard_cost  product_first_sold_date  
count   19803.000000             19803.000000  
mean      556.046951             38199.776549  
std       405.955660              2875.201110  
min         7.210000             33259.000000  
25%      

1_2. Quantitative data about prices seem reasonable. Let check completeness of data

In [163]:
#Check Completeness

# Sum nulls whenever True it will sum up and give total missing values
print(transactions.isnull().sum())

transaction_id               0
product_id                   0
customer_id                  0
transaction_date             0
online_order               360
order_status                 0
brand                      197
product_line               197
product_class              197
product_size               197
list_price                   0
standard_cost              197
product_first_sold_date    197
dtype: int64


We lack info about online_order for 360 transactions, and 197 missing data for brand, product_line etc.
    Is this missing data overlap?

In [164]:
#Check Completeness further
#combine cases where online_order and brand columns are null
print("\n quantity of transactions where any info is missing {} \n".format(len(transactions[(transactions.online_order.isnull()) | (transactions.brand.isnull())])))

# look into 197 rows of missing data
print('\n',transactions[(transactions.brand.isnull())])


 quantity of transactions where any info is missing 555 


        transaction_id  product_id  customer_id transaction_date  online_order  \
136               137           0          431       2017-09-23           0.0   
159               160           0         3300       2017-08-27           0.0   
366               367           0         1614       2017-03-10           0.0   
406               407           0         2559       2017-06-14           1.0   
676               677           0         2609       2017-07-02           0.0   
...               ...         ...          ...              ...           ...   
19340           19341           0          443       2017-12-26           1.0   
19383           19384           0         2407       2017-06-11           0.0   
19793           19794           0         2860       2017-01-13           0.0   
19859           19860           0         2468       2017-06-24           1.0   
19871           19872           0           61  

In [165]:
# Check Consistency
#Check for first and last transaction date
print('first order made on {}. '.format(transactions.transaction_date.min()),'Last order made on {}'.format(transactions.transaction_date.max()))

first order made on 2017-01-01 00:00:00.  Last order made on 2017-12-30 00:00:00


1_3. online_order info just carries info about whether the transaction was done online or in store, i assume considering small amount of missing data, its not so critical for analytical purposes for this case and 360 rows can be filled with either of choices 1 or 0.<br> 
    Other 197 missing data in 6 columns brand, product_line, product_class, product_size, standard_cost, product_first_sold_date are carrying important information, and other way than just removing these rows i can't suggest. I might assume it could be some kind of service rather than product done both online and in store. Transaction_dates are consistent, all orders are made within 1 year.  

In [168]:
#delete variable in case this lines of code will be run several times
#del(transactions_new)

#Solve inConsistency
# remove rows where brand and other columns are missing
transactions_new = transactions.dropna(subset = ['brand'])

# fill with 0 rows where online_order is NaN
transactions_new.fillna( 0, inplace = True)

In [169]:
#Check Validity for category columns
cat_columns = ['order_status', 'brand', 'product_line', 'product_class', 'product_size']
for category in cat_columns:
    print(category,' categories: ',transactions_new[category].unique())

order_status  categories:  ['Approved' 'Cancelled']
brand  categories:  ['Solex' 'Trek Bicycles' 'OHM Cycles' 'Norco Bicycles' 'Giant Bicycles'
 'WeareA2B']
product_line  categories:  ['Standard' 'Road' 'Mountain' 'Touring']
product_class  categories:  ['medium' 'low' 'high']
product_size  categories:  ['medium' 'large' 'small']


In [170]:
#Check Uniqueness
#Check keys of database transaction_id
print(transactions_new.transaction_id.duplicated().value_counts())

False    19803
Name: transaction_id, dtype: int64


In [235]:
# Lets see how many customers made transactions
tr_custid = np.array(transactions_new.customer_id.sort_values(ascending = True).unique())
print('how many customers made orders out of 4000 (or 4003): ',len(tr_custid), ' customers')

how many customers made orders out of 4000 (or 4003):  3494  customers


1_4. Summary according to Data-Quality Dimensions:<br>
        Accuracy:<br>
            no issues<br>
        Completeness:<br>
            online_order: missing for 360 transactions; brand, product_line etc: 197 rows of missing data.<br>
            197 rows where brand, product_line were missing were deleted; 360 rows of online_order info was filled with 0 <br>
        Consistency:<br>
            all data is consistent <br>
        Currency:<br>
            not applicable for current project<br>
        Relevancy:<br>
            no unnecessary info<br>
        Validity:<br>
            invalid data in catgorical columns were not found<br>
        Uniqueness:<br>
            no issues, transaction dataset contains unique transaction_id

In [171]:
#print quatity of NaN info in any of columns
print(transactions_new.isnull().sum())

#print new size of dataframe
print('\n Size of new data frame {}'.format(transactions_new.shape))

transaction_id             0
product_id                 0
customer_id                0
transaction_date           0
online_order               0
order_status               0
brand                      0
product_line               0
product_class              0
product_size               0
list_price                 0
standard_cost              0
product_first_sold_date    0
dtype: int64

 Size of new data frame (19803, 13)


2_1. Time to start looking inside CustomerDemographic data. By describe() and summing NaN we'll see missing info in each columns if any

In [238]:
# See the summary of customer demographics
#Find quantity of customers in database
print("customer quantity {} \n".format(len(customerdemo)))

#Compute quantitative summary for numerical columns
print(customerdemo.describe())

#Caclulate missing data in columns
print('\n',customerdemo.isnull().sum())

#overview for data first 5 rows
print('\n',customerdemo.head())

customer quantity 4000 

       customer_id  past_3_years_bike_related_purchases       tenure
count  4000.000000                          4000.000000  3913.000000
mean   2000.500000                            48.890000    10.657041
std    1154.844867                            28.715005     5.660146
min       1.000000                             0.000000     1.000000
25%    1000.750000                            24.000000     6.000000
50%    2000.500000                            48.000000    11.000000
75%    3000.250000                            73.000000    15.000000
max    4000.000000                            99.000000    22.000000

 customer_id                              0
first_name                               0
last_name                              125
gender                                  88
past_3_years_bike_related_purchases      0
DOB                                     87
job_title                              506
job_industry_category                  656
wealth_s

In [239]:
#Check Accuracy
#check DOB
print(customerdemo.DOB.dropna().sort_values(ascending=True))

#solve inAccuracy
#find out the mean DOB
print(customerdemo.DOB.mean())

#find index of impossible BOD
print(customerdemo[customerdemo.DOB==customerdemo.DOB.min()].index)

#store index
old_index=customerdemo[customerdemo.DOB==customerdemo.DOB.min()].index

#substitute with mean BOD
customerdemo.loc[old_index,'DOB']=customerdemo.DOB.mean()
print(customerdemo.loc[old_index,'DOB'])

719    1931-10-23
1091   1935-08-22
3409   1940-09-22
2412   1943-08-11
657    1944-01-24
          ...    
421    2002-01-06
2857   2002-01-09
3434   2002-01-15
1887   2002-01-26
65     2002-03-11
Name: DOB, Length: 3913, dtype: datetime64[ns]
1977-07-25 11:08:39.754991296
Int64Index([719], dtype='int64')
719   1977-07-25 11:08:39.754991296
Name: DOB, dtype: datetime64[ns]


In [240]:
#Check Consistency
print('Doesnt own a car but tenure > 0 :',len(customerdemo[(customerdemo.owns_car == 'No') & (customerdemo.tenure > 0) ]))

Doesnt own a car but tenure > 0 : 1939


In [241]:
#Check Relevancy
#check default
print(customerdemo.default.dropna())

0                                                      "'
1                            <script>alert('hi')</script>
2                                     2018-02-01 00:00:00
3       () { _; } >_[$($())] { touch /tmp/blns.shellsh...
4                                                     NIL
                              ...                        
3994                                                   á 
3995                                                 -100
3996                                             â¦testâ§
3998                               Â¡â¢Â£Â¢âÂ§Â¶â¢ÂªÂºââ 
3999                                                  0/0
Name: default, Length: 3698, dtype: object


In [242]:
#Check Validity for category columns
cat_columns = ['gender','wealth_segment', 'job_industry_category', 'deceased_indicator','owns_car']
for category in cat_columns:
    print(category,' categories: ',customerdemo[category].unique())

gender  categories:  ['Female' 'Male' nan]
wealth_segment  categories:  ['Mass Customer' 'Affluent Customer' 'High Net Worth']
job_industry_category  categories:  ['Health' 'Financial Services' 'Property' 'IT' nan 'Retail' 'Argiculture'
 'Manufacturing' 'Telecommunications' 'Entertainment']
deceased_indicator  categories:  ['N' 'Y']
owns_car  categories:  ['Yes' 'No']


In [243]:
#Solve gender categories bu mapping with only 2 gender Female and Male and NaN where U
mapping = {'F':'Female','Female':'Female','Femal':'Female','M':'Male', 'Male':'Male', 'U':np.nan}
customerdemo['gender']=customerdemo['gender'].map(mapping)
print(customerdemo['gender'].unique())

['Female' 'Male' nan]


In [244]:
#Check Uniqueness
#Check keys customer_id for database
print(customerdemo[customerdemo.customer_id.duplicated()==True])

#Check consistency of customer_id it supposed to be from 1 to 4000 total of 4000 counts:
x=np.linspace(1,4000,4000).astype('int')
customerdemo_list = []
custdemoid = np.array(customerdemo.customer_id.sort_values(ascending = True))

#iterate over 4000 to find inconsistency in customer_id's
for i in x:         
    if i not in custdemoid:
        customerdemo_list.append(i)
        
#Print resulting lis, if empty then no inconsistency        
print('\ncustomer_demo customers with ids: ', customerdemo_list)

Empty DataFrame
Columns: [customer_id, first_name, last_name, gender, past_3_years_bike_related_purchases, DOB, job_title, job_industry_category, wealth_segment, deceased_indicator, default, owns_car, tenure]
Index: []

customer_demo customers with ids:  []


2_2. Summary according to Data-Quality Dimensions:<br>
        Accuracy:<br>
            DOB:contains very old man<br>
        Completeness:<br>
            DOB, job_title and job_industry_cat columns: missing data, meaning lack of personal info hence less advert possibilities.<br>
            last_name: missing, however customer_id is present, not a big issue.<br>
        Consistency:<br>
            owns_car vs tenure: 3rd row shows customer doesn't posses car however tenure columns shows values. Most probably i dont know what tenure really means in this database, since 1939 rows contain tenure values while not having a car <br>
        Currency:<br>
            not applicable for current project<br>
        Relevancy:<br>
            default: contains unnecessary info<br>
        Validity:<br>
            gender have different categories e.g.: F, Female, Male...<br>
        Uniqueness:<br>
            no issues, customerdemographics base contains unique customer_id

In [245]:
#Check the shape of final dataframe
print('\n',customerdemo.shape)

#store final dataset to new variable

customerdemo_new = customerdemo


 (4000, 13)


3_1. Finally we check the customer address database

In [161]:
# See the quantitative summary of customeradd
print("customeraddress quantity {} \n".format(len(customeradd)))
print(customeradd.describe())
#Caclulate missing data in columns
print('\n',customeradd.isnull().sum())
print('\n',customeradd.head())

customeraddress quantity 3999 

       customer_id     postcode  property_valuation
count  3999.000000  3999.000000         3999.000000
mean   2003.987997  2985.755939            7.514379
std    1154.576912   844.878364            2.824663
min       1.000000  2000.000000            1.000000
25%    1004.500000  2200.000000            6.000000
50%    2004.000000  2768.000000            8.000000
75%    3003.500000  3750.000000           10.000000
max    4003.000000  4883.000000           12.000000

 customer_id           0
address               0
postcode              0
state                 0
country               0
property_valuation    0
dtype: int64

    customer_id              address  postcode state    country  \
0            1   060 Morning Avenue      2016   NSW  Australia   
1            2  6 Meadow Vale Court      2153   NSW  Australia   
2            4   0 Holy Cross Court      4211   QLD  Australia   
3            5  17979 Del Mar Point      2448   NSW  Australia   
4        

In [155]:
#Check Validity for category columns
cat_columns = ['country', 'state']
for category in cat_columns:
    print(category,' categories: ',customeradd[category].unique())

country  categories:  ['Australia']
state  categories:  ['New South Wales' 'QLD' 'VIC' 'NSW' 'Victoria']


In [160]:
#Solve inValid/inConsistent data
#use map to substitute 'New South Wales' to 'NSW', and 'Victoria' to 'VIC'
mapping = {'New South Wales':'NSW', 'Victoria':'VIC', 'QLD':'QLD', 'NSW':'NSW', 'VIC':'VIC'}
customeradd['state'] = customeradd['state'].map(mapping)
customeradd['state'].unique()

array(['NSW', 'QLD', 'VIC'], dtype=object)

In [149]:
#Check Uniqueness
#Check keys customer_id for database
print(customeradd[customeradd.customer_id.duplicated()==True])

Empty DataFrame
Columns: [customer_id, address, postcode, state, country, property_valuation]
Index: []


In [237]:
# Check Completness based on Uniqueness:
#create empty lists to see what customer_id lacks in compare to customer_demographics dataset and vice versa:
x=np.linspace(1,4000,4000).astype('int')
custid = np.array(customeradd.customer_id.sort_values(ascending = True))
missing_list = []
extra_list = []

#iterate over 4000 to check what is in customer_id and what are extra id's that cant be found in customer_demographics dataset
for i in x:
    if i not in custid:
        missing_list.append(i)
for i in custid:
    if i not in x:
        extra_list.append(i)

print('list of missing address info abt customers with ids: ', missing_list)
print('list of extra address info abt customers with ids not present customerdemographics dataset: ',extra_list)
print(custid)

#store final set to new variable
customeradd_new = customeradd

list of missing address info abt customers with ids:  [3, 10, 22, 23]
list of extra address info abt customers with ids not present customerdemographics dataset:  [4001, 4002, 4003]
[   1    2    4 ... 4001 4002 4003]


3_2. Summary according to Data-Quality Dimensions:<br>
        Accuracy:<br>
            no issues<br>
        Completeness:<br>
            no missing data for given dataset, however cusomer_id lacks customer_id [3 10 22 23] and having extra ids [4001, 4002, 4003] can't be found in customer_demographics <br>
        Consistency:<br>
            no issues <br>
        Currency:<br>
            not applicable for current project<br>
        Relevancy:<br>
            no issues<br>
        Validity:<br>
            state: several notation for same state, was dealt with mapping to 3 letter notation <br>
        Uniqueness:<br>
            no issues in uniquenss, customeraddress base contains unique customer_id, however problems with completness

4. Summary and export final cleaned data back to excel

In [229]:
# lets summarize
    #We have 3 datasets:
        #transactions: have primary key which is transaction_id column and foreign key which is customer_id column
        #customerdemo: have primary key which is customer_id column from 1 to 4000, total 4000 counts
        #customeraddress: have primary key which is customer_id column from 1 to 4003 with some missing and extra id's

In [248]:
# we can join 3 datasets, 2 (customeraddress & customerdemo) on outer join since we have some unoverlapping id's,
combined_data = pd.merge(left = customerdemo_new, right = customeradd_new, left_on = 'customer_id', right_on = 'customer_id',how = 'outer')

# and finally left join (customers_combined info on the left) on transactions, sutomers that did not order anything will be in database:
combined_data = pd.merge(left = combined_data, right = transactions_new, left_on = 'customer_id', right_on = 'customer_id',how = 'left')
print(len(combined_data),'\n', combined_data)

20310 
        customer_id first_name  last_name  gender  \
0                1    Laraine  Medendorp  Female   
1                1    Laraine  Medendorp  Female   
2                1    Laraine  Medendorp  Female   
3                1    Laraine  Medendorp  Female   
4                1    Laraine  Medendorp  Female   
...            ...        ...        ...     ...   
20305         3999  Patrizius        NaN    Male   
20306         4000      Kippy    Oldland    Male   
20307         4001        NaN        NaN     NaN   
20308         4002        NaN        NaN     NaN   
20309         4003        NaN        NaN     NaN   

       past_3_years_bike_related_purchases        DOB             job_title  \
0                                     93.0 1953-10-12   Executive Secretary   
1                                     93.0 1953-10-12   Executive Secretary   
2                                     93.0 1953-10-12   Executive Secretary   
3                                     93.0 1953-10-

In [249]:
# Create a Pandas Excel writer using XlsxWriter as the engine.
writer = pd.ExcelWriter('KPMG_new.xlsx', engine='xlsxwriter')

# Write each dataframe to a different worksheet.
transactions_new.to_excel(writer, sheet_name='transactions')
customerdemo_new.to_excel(writer, sheet_name='customerdemographics')
customeradd_new.to_excel(writer, sheet_name='customeraddress')
combined_data.to_excel(writer, sheet_name='combined')


# Close the Pandas Excel writer and output the Excel file.
writer.save()